In [ ]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

In [ ]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key=os.getenv("GROQ_API_KEY"), 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-43537")
page_data = loader.load().pop().page_content

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})

str

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer II, ITC',
 'experience': '3-6 years',
 'skills': ['Fullstack development',
  'Front-end technologies (React)',
  'Backend development (Node.js OR Java)',
  'Microservice architecture',
  'RESTful APIs',
  'GraphQL services',
  'Cloud environments (AWS, Azure, or GCP)',
  'Test-Driven Development (TDD)',
  'JUnit, Jest, or Mocha',
  'Relational and NoSQL databases (e.g., MySQL, MongoDB, DynamoDB)',
  'CI/CD pipelines',
  'Version control',
  'Agile methodologies',
  'UI/UX best practices',
  'Performance optimization',
  'Responsive design principles'],
 'description': 'As a Software Engineer 2 - Fullstack, you will be part of a dynamic team driving the creation of both backend and frontend solutions, delivering powerful, high-performance applications across digital and in-store platforms. You will be passionate about creating seamless user experiences, efficient backend processes, and quality code.'}

In [7]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [10]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

C:\Users\Ashwini\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:55<00:00, 1.50MiB/s]  


In [ ]:
job = json_res
job['skills']

['Fullstack development',
 'Front-end technologies (React)',
 'Backend development (Node.js OR Java)',
 'Microservice architecture',
 'RESTful APIs',
 'GraphQL services',
 'Cloud environments (AWS, Azure, or GCP)',
 'Test-Driven Development (TDD)',
 'JUnit, Jest, or Mocha',
 'Relational and NoSQL databases (e.g., MySQL, MongoDB, DynamoDB)',
 'CI/CD pipelines',
 'Version control',
 'Agile methodologies',
 'UI/UX best practices',
 'Performance optimization',
 'Responsive design principles']

In [14]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])

In [15]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Full-Stack Development Services for Your ITC Project

Dear Hiring Manager,

I came across your job posting for a Software Engineer II, ITC, and I'm excited to introduce AtliQ, an AI & Software Consulting company that can help you achieve your project goals. With our expertise in full-stack development, we can drive the creation of both backend and frontend solutions, delivering powerful, high-performance applications across digital and in-store platforms.

Our team is well-versed in the technologies you're looking for, including:

* Full-stack development
* Front-end technologies (React)
* Backend development (Node.js and Java)
* Microservice architecture
* RESTful APIs and GraphQL services
* Cloud environments (AWS, Azure, and GCP)
* Test-Driven Development (TDD) with JUnit, Jest, and Mocha
* Relational and NoSQL databases (e.g., MySQL, MongoDB, and DynamoDB)
* CI/CD pipelines and version control
* Agile methodologies and UI/UX best practices

We've successfully delive